# Edit Your Expectation Suite
Use this notebook to recreate and modify your expectation suite:

**Expectation Suite Name**: `aermokhin.ods_billing.warning`

We'd love it if you **reach out to us on** the [**Great Expectations Slack Channel**](https://greatexpectations.io/slack)

In [ ]:
import datetime
import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.checkpoint import LegacyCheckpoint
from great_expectations.data_context.types.resource_identifiers import ValidationResultIdentifier

context = ge.data_context.DataContext()

# Feel free to change the name of your suite here. Renaming this will not
# remove the other one.
expectation_suite_name = "aermokhin.ods_billing.warning"
suite = context.get_expectation_suite(expectation_suite_name)
suite.expectations = []

batch_kwargs = {'data_asset_name': 'aermokhin.ods_billing', 'datasource': 'gp', 'limit': 1000, 'schema': 'aermokhin', 'table': 'ods_billing'}
batch = context.get_batch(batch_kwargs, suite)
batch.head()

## Create & Edit Expectations

Add expectations by calling specific expectation methods on the `batch` object. They all begin with `.expect_` which makes autocompleting easy using tab.

You can see all the available expectations in the **[expectation glossary](https://docs.greatexpectations.io/en/latest/reference/glossary_of_expectations.html?utm_source=notebook&utm_medium=create_expectations)**.

### Table Expectation(s)

In [ ]:
batch.expect_table_row_count_to_be_between(max_value=1100, min_value=900)

In [ ]:
batch.expect_table_column_count_to_equal(value=6)

In [ ]:
batch.expect_table_columns_to_match_ordered_list(column_list=['user_id', 'billing_period', 'service', 'tariff', 'sum', 'created_at'])

### Column Expectation(s)

#### `tariff`

In [ ]:
batch.expect_column_values_to_not_be_null(column='tariff')

In [ ]:
batch.expect_column_distinct_values_to_be_in_set(column='tariff', value_set=['Gigabyte', 'Maxi', 'Megabyte', 'Mini'])

In [ ]:
batch.expect_column_kl_divergence_to_be_less_than(column='tariff', partition_object={'values': ['Gigabyte', 'Maxi', 'Megabyte', 'Mini'], 'weights': [0.254, 0.254, 0.266, 0.226]}, threshold=0.6)

#### `billing_period`

In [ ]:
batch.expect_column_values_to_not_be_null(column='billing_period')

In [ ]:
batch.expect_column_value_lengths_to_be_between(column='billing_period', min_value=1)

#### `created_at`

In [ ]:
batch.expect_column_values_to_not_be_null(column='created_at')

In [ ]:
batch.expect_column_values_to_be_between(column='created_at', max_value='2015-12-30 00:00:00', min_value='2012-01-02 00:00:00', parse_strings_as_datetimes=True)

## Save & Review Your Expectations

Let's save the expectation suite as a JSON file in the `great_expectations/expectations` directory of your project.
If you decide not to save some expectations that you created, use [remove_expectation method](https://docs.greatexpectations.io/en/latest/autoapi/great_expectations/data_asset/index.html?highlight=remove_expectation&utm_source=notebook&utm_medium=edit_expectations#great_expectations.data_asset.DataAsset.remove_expectation).

Let's now rebuild your Data Docs, which helps you communicate about your data with both machines and humans.

In [ ]:
batch.save_expectation_suite(discard_failed_expectations=False)

results = LegacyCheckpoint(
    name="_temp_checkpoint",
    data_context=context,
    batches=[
        {
          "batch_kwargs": batch_kwargs,
          "expectation_suite_names": [expectation_suite_name]
        }
    ]
).run()
validation_result_identifier = results.list_validation_result_identifiers()[0]
context.build_data_docs()
context.open_data_docs(validation_result_identifier)